# A quick tour of model selection, combination, and regularization

* Vivek Srikrishnan and James Doss-Gollin
* Keller Group Meeting
* Thursday 16 July 2020

Key references:

* Piironen & Vehtari (2017)
* Gelman et al BDA3 (2014)
* Heinze et al 2018

## The challenge

We want to make probabilistic predictions about **unobserved** data $\tilde{y}$.
This is hard because Earth systems are:

* high-dimensional multi-scale
* nonlinear / complex

To approximate the true system, we come up with a **model space** $\mathcal{M}$ defining a family of candidate models, then use them to make predictions.

## Some background theory

_OMITTED FROM SLIDES BUT KEPT HERE FOR REFERENCE_

Recall:

$$
D_\text{KL} (P \parallel Q) = \sum_{x \in \mathcal{X}} P(x) \log \left[ \frac{P(x)}{Q(x)} \right]
$$

One interpretation of $D_\text{KL} (P \parallel Q)$ is the measure of information gained by revising one's beliefes from the prior distribution $Q$ to the posterior distribution $P$.
Another interpretation is the amount of information lost when $Q$ is used to approximate $P$.
Note that for continuous RVs the above sum can be written as an integral.

### Measures of predictive accuracy

Predictive performance of a model defined in terms of a utility function $u(M, \tilde{y})$.
Commonly used: log predictive density: 
$$
\log p(\tilde{y} | D, M).
$$

Future observations $\tilde{y}$ are unknown, so we must approach it in expectation:
$$
\overline{u}(M) = \mathbb{E}\left[ \log p(\tilde{y} | D, M) \right] = \int p_t(\tilde{y}) \log [(\tilde{y} | D, M) d\tilde{y}
$$
where $p_t(\tilde{y})$ is the true data generating distribution (unknown!)

This has nice properties: maximizing $\overline{u}(M)$ is equivalent to minimizing KL divergence from candidate model $p(\tilde{y} | D, M)$ to true data distribution $p_t(\tilde{y})$

### In practice we work with posterior estimates

We don't know the true distribution $\theta$ so we have to approximate it.
The log pointwise predictive density is
$$
\begin{align}
\text{lppd} &= \log \prod_{i=1}^N p_\text{post}(y_i) = \sum_{i=1}^N \log \int p(y_i | \theta) p_\text{post} (\theta) d \theta \\
&\approx \sum_{i=1}^N \log \left[ \frac{1}{S} \sum_{i=1}^S p(y_i | \theta^s) \right]
\end{align}
$$
where we have approximated the posterior with $S$ simulations from the posterior (eg, using MCMC).

Key point:

> the LPPD of observed data $y$ is an overestimate of the expected LPPD for future data. Thus tools will start with our approximate form and then derive some correction.

## Model combination

We could try to sample from the model space.
If we have an exhaustive list of candidate models $\{ M_\ell \}_{\ell=1}^L$, then the distribution over the *model space* is given by
$$
p(M | D) \propto p(D | M) p(M)
$$
and we can average over them
$$
p(\tilde{y} | D) = \sum_{\ell=1}^L p(\tilde{y}|D, M_\ell) p(M_\ell | D)
$$
strictly speaking this is an $\mathcal{M}$-closed assumption but in practice this is often not a super critical assumption

### MAP

Alternatively, choose the model with the highest posterior probability (i.e., the "best" model).
For $L=2$ we can see the close analogy to the Bayes Factor:
$$
K = \frac{\Pr(D|M_1)}{\Pr(D|M_2)}
= \frac{\int \Pr(\theta_1|M_1)\Pr(D|\theta_1,M_1)\,d\theta_1}
{\int \Pr(\theta_2|M_2)\Pr(D|\theta_2,M_2)\,d\theta_2}
= \frac{\Pr(M_1|D)}{\Pr(M_2|D)}\frac{\Pr(M_2)}{\Pr(M_1)}.
$$
Remember, though, that the candidate model set can be arbitrary -- hence the many problems with significance testing!

## Generalizing LPPD

### Cross-Validation

* We can use the sample data $D$ as a proxy for $p_t(\tilde{y}$
* Estimating $\mathbb{E}[u(M, \tilde{y})]$  using training data $D$ biases generalization performance (overfitting)
* Divide the data into $K$ subsets; for $i=1, \ldots, K$ hold out the $i$th subset, use it for validation, and use the rest for traning
* Small $K$ also induces bias -- $K=N$ is ideal but expensive so $K=10$ often used
* Approximations to LOO ($K=N$) will be discussed later

### AIC Criterion

If our inference on the parameters is summarized by a point estimate $\hat{\theta}$ (e.g., the maximum likelihood estimate) then out of sample predictive accuracy is defined by
$$
\text{elpd}_\hat{\theta} = \mathbb{E}_f \left[ \log p(\tilde{y} | \hat{\theta}(y)) \right]
$$
If the model estimates $k$ parameters, and if they are assumed asymptotically normal (ie a normal linear model with known variance and uniofrm prior) then fitting $k$ parameters will increase the predictive accuracy by chance alone:
$$
\hat{\text{elpd}}_\text{AIC} = \log p(y | \hat{\theta}_\text{mle}) - k
$$
Thus we can define
$$
\text{AIC} = 2 k - 2 \ln \hat{\mathcal{L}}
$$
and select the model that minimizes it.

_For complicated models, what is $k$?_
There are formula to approximate effective number of parameters.
Note that AIC asssumes residuals are independent given $\hat{\theta}$

### DIC Criterion

1. Start with AIC
1. Replace $\hat{\theta}_\text{mle}$ by posterior mean $\hat{\theta}_\text{Bayes} = \mathbb{E}[\theta | y]$
1. Replace $k$ by a data-based bias correction; there are different forms

$$
\hat{\text{elpd}}_\text{DIC} = \log p(y | \hat{\theta}_\text{Bayes}) - p_\text{DIC}
$$
where $p_\text{DIC}$ is derived from assumptions about the effective number of parameters.
The quantity
$$
\text{DIC} = -2 \log p(y | \hat{\theta}_\text{Bayes}) + 2 p_\text{DIC}
$$
can be assigned to each model, and the model with lowest DIC chosen.
Note that DIC asssumes residuals are independent given $\hat{\theta}$

### WAIC

More fully Bayesian information criterion and _can be viewed as approximation to cross-validation_.

Define the bias correction penalty:
$$
p_\text{WAIC2} = \sum_{i=1}^N \mathbb{V}_\text{post} \left[ \log p(y_i | \theta) \right]
$$
and use
$$
\hat{\text{elppd}}_\text{WAIC} = \text{lppd} - p_\text{WAIC}
$$

WAIC is an approximation to the number of 'unconstrained' parameters in the model:

* a parameter counts as 1 if it is estimated with no constraints or prior information
    * param counts as 0 if it is fully constrained by the prior
    * param gives intermediate value if both the data and prior distributions are informative.

WAIC averages over posterior, which is good.
_BDA3 recommends WAIC over AIC and DIC but it requires partioning data into $n$ pieces_.

### Schwarz criterion / "Bayesian" information criterion (BIC, SBC, SIC, SBIC)

Goal: approximate marginal probability of the data $p(y)$ (this is different)

Assuming the existence of a true model ($\mathcal{M}-closed$), the model that minimizes BIC converges to the "true" model.
$$
\text{BIC} = k \ln (n) - 2 \ln \hat{\mathcal{L}}
$$
where
$$
\hat{\mathcal{L}}= \max_\theta p(x | \theta, M)
$$
and where $k$ is the number of model parameters.
The BIC can be viewed as a rough approximation to the Bayes factor (Kass and Raftery 1995).

### Significance criteria

Use Null Hypothesis Significance Testing (NHST) to decide whether to include a variable.
For example, should we add a trend term in our regression?

1. Form a null hypothesis: $\beta = 0$
1. Test statistics $\Rightarrow$ $p$-value
1. If $p < \alpha$ then use $M_2$ else use $M_1$

Note that

* This is equivalent to Bayes factor.
* Still assumes existence of a true model (hence the many problems with NHST)

**This is widely used in practice, often without justification**

## Reference model approach

* Instead of trying to find a MAP model from $\mathcal{M}$, approximate a "reference" model $M_*$
* IE: what is the best emulator, conditional on believing the reference model?

### Reference predictive method

We can estimate the utilities of the candidate models by replacing $p_t(\tilde{y})$ by $p(\tilde{y} | D, M_*)$:
$$
\overline{u}_\text{ref}(M) = \frac{1}{N} \sum_{i=1}^N \int \underbrace{p(\tilde{y} | x_i, D, M_*)}_{\approx p_t} \log p(\tilde{y} | x_i, D, M) d \tilde{y}
$$

Maximizing the reference utility is equivalent to minimizing $D_\text{KL} (M_* \parallel M)$.

### Projection predictive method

Goal: project the information in the posterior of the reference model $M_*$ onto the candidate models.

* Parameters of candidate models are determined by the reference model, not by data
* Only reference model needs to be fit / calibrated

Given parameters of the reference model $\theta^*$, the parameters of model $M$ is
$$
\theta^\perp = \arg \min_\theta \frac{1}{N} \sum_{i=1}^N \text{KL} \left[ p(\tilde{y} | x_i, \theta^*, M_*) \parallel p(\tilde{y} | x_i, \theta^\perp, M) \right]
$$
which can be approximated by samples and used to set a rule for choosing a model.

## Conclusions: No Magic Here

* Regularization: fit models that are more robust in the first place
* "In a sparse-data setting, a poor choice of prior distribution can lead to weak inferences and poor predictions." BDA3. Analagous: equifinality.
* "Informative prior distributions and hierarchical structures tend to reduce the amount of overfitting, compared to what would happen under simple least squares or maximum likelihood estimation."

A complement to automatic variable selection / combination is to start with a simple model and expand iteratively in a [principled](https://github.com/betanalpha/jupyter_case_studies/blob/master/principled_bayesian_workflow/principled_bayesian_workflow.ipynb) fashion using posterior predictive checks and domain expertise, expanding the model only where clear deficiencies are identified

### Topics deferred

* Stacking -- a clever way to do model combination in the $\mathcal{M}$-open case
* Regularization (LASSO, Ridge, sparse priors, hierarchical structure) can push some coefficients towards group mean or zero

## References / read more

* Gelman, A., & Loken, E. (2013, November 14). The garden of forking paths: Why multiple comparisons can be a problem, even when there is no “fishing expedition” or “p-hacking” and the research hypothesis …. Retrieved from http://www.stat.columbia.edu/~gelman/research/unpublished/p_hacking.pdf
* Heinze, G., Wallisch, C., & Dunkler, D. (2018). Variable selection – A review and recommendations for the practicing statistician. Biometrical Journal, 60(3), 431–449. https://doi.org/10.1002/bimj.201700067
* Kass, R. E., & Raftery, A. E. (1995). Bayes Factors. Journal of the American Statistical Association, 90(430), 773–795. https://doi.org/10.1080/01621459.1995.10476572
* Navarro, D. J. (2018). Between the Devil and the Deep Blue Sea: Tensions Between Scientific Judgement and Statistical Model Selection. Computational Brain & Behavior. https://doi.org/10.1007/s42113-018-0019-z
* Piironen, J., & Vehtari, A. (2017). Comparison of Bayesian predictive methods for model selection. Statistics and Computing, 27(3), 711–735. https://doi.org/10.1007/s11222-016-9649-y
* Vehtari, A., Gelman, A., & Gabry, J. (2017). Practical Bayesian Model Evaluation Using Leave-One-out Cross-Validation and WAIC. Statistics and Computing, 27(5), 1413–1432. https://doi.org/10.1007/s11222-016-9696-4
* Yao, Y., Vehtari, A., Simpson, D., & Gelman, A. (2018). Using Stacking to Average Bayesian Predictive Distributions. Bayesian Analysis. https://doi.org/10.1214/17-BA1091
